In [11]:
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import os
from azure.ai.projects import AIProjectClient
from azure.core.credentials import AzureKeyCredential 
from azure.ai.agents.models import FunctionTool, ToolSet, ListSortOrder
from azure.ai.agents.models import AgentsNamedToolChoice, AgentsNamedToolChoiceType, FunctionName
from azure.search.documents.agent import KnowledgeAgentRetrievalClient
from azure.search.documents.agent.models import KnowledgeAgentRetrievalRequest, KnowledgeAgentMessage, KnowledgeAgentMessageTextContent, KnowledgeAgentIndexParams


In [ ]:


# load_dotenv(override=True) # take environment variables from .env.

# # The following variables from your .env file are used in this notebook
# project_endpoint = os.environ["PROJECT_ENDPOINT"]
# agent_model = os.getenv("AGENT_MODEL", "gpt-4.1-mini")
# endpoint = os.environ["AZURE_SEARCH_ENDPOINT"]
# credential = DefaultAzureCredential()
# token_provider = get_bearer_token_provider(credential, "https://search.azure.com/.default")
# index_name = os.getenv("AZURE_SEARCH_INDEX", "earth_at_night")
# azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
# azure_openai_gpt_deployment = os.getenv("AZURE_OPENAI_GPT_DEPLOYMENT", "gpt-4.1-mini")
# azure_openai_gpt_model = os.getenv("AZURE_OPENAI_GPT_MODEL", "gpt-4.1-mini")
# azure_openai_embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT", "text-embedding-3-large")
# azure_openai_embedding_model = os.getenv("AZURE_OPENAI_EMBEDDING_MODEL", "text-embedding-3-large")
# agent_name = os.getenv("AZURE_SEARCH_AGENT_NAME", "earth-search-agent")

In [ ]:

# load_dotenv(override=True) # take environment variables from .env.


# project_endpoint = os.environ["PROJECT_ENDPOINT"]
# agent_model = os.getenv("AGENT_MODEL", "gpt-4.1-mini")
# endpoint = os.environ["AZURE_SEARCH_ENDPOINT"]
# credential = DefaultAzureCredential(managed_identity_client_id  = "4367ceb1-6348-41d1-92a1-b5583f2ec5de") 
# project_credential = credential # Replace with your managed identity client ID if needed
# token_provider = get_bearer_token_provider(credential, "https://search.azure.com/.default")
# index_name = os.getenv("AZURE_SEARCH_INDEX", "earth_at_night")
# azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
# azure_openai_gpt_deployment = os.getenv("AZURE_OPENAI_GPT_DEPLOYMENT", "gpt-4.1-mini")
# azure_openai_gpt_model = os.getenv("AZURE_OPENAI_GPT_MODEL", "gpt-4.1-mini")
# azure_openai_embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT", "text-embedding-3-large")
# azure_openai_embedding_model = os.getenv("AZURE_OPENAI_EMBEDDING_MODEL", "text-embedding-3-large")
# agent_name = os.getenv("AZURE_SEARCH_AGENT_NAME", "earth-search-agent")

In [12]:


load_dotenv(override=True) # take environment variables from .env.

# The following variables from your .env file are used in this notebook
project_endpoint = os.environ["PROJECT_ENDPOINT"]
agent_model = os.getenv("AGENT_MODEL", "gpt-4.1-mini")
endpoint = os.environ["AZURE_SEARCH_ENDPOINT"]
new_credential = AzureKeyCredential("Z9qrhCZDgy2NYcqxdaIAzbEZ8IeSrNnsQluoogM6N0AzSeD1Fr3z")
credential = DefaultAzureCredential()
# credential = DefaultAzureCredential(managed_identity_client_id  = "4367ceb1-6348-41d1-92a1-b5583f2ec5de")
token_provider = get_bearer_token_provider(credential, "https://search.azure.com/.default")
index_name = os.getenv("AZURE_SEARCH_INDEX", "earth_at_night")
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_openai_gpt_deployment = os.getenv("AZURE_OPENAI_GPT_DEPLOYMENT", "gpt-4.1-mini")
azure_openai_gpt_model = os.getenv("AZURE_OPENAI_GPT_MODEL", "gpt-4.1-mini")
azure_openai_embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT", "text-embedding-3-large")
azure_openai_embedding_model = os.getenv("AZURE_OPENAI_EMBEDDING_MODEL", "text-embedding-3-large")
agent_name = os.getenv("AZURE_SEARCH_AGENT_NAME", "test-agent")

In [13]:
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential
import os

from azure.identity import AzureCliCredential
from azure.identity import VisualStudioCodeCredential

credential = DefaultAzureCredential()
token = credential.get_token("https://management.azure.com/.default")
print("Using identity:", token.token[:20], "...")


Using identity: eyJ0eXAiOiJKV1QiLCJh ...


In [14]:
from azure.identity import DefaultAzureCredential
from azure.mgmt.resource import SubscriptionClient

credential = DefaultAzureCredential()
sub_client = SubscriptionClient(credential)

for sub in sub_client.subscriptions.list():
    print(f"Authenticated to Subscription: {sub.display_name} ({sub.subscription_id})")


Authenticated to Subscription: MCAPS-Hybrid-REQ-76999-2024-mtalreja (eb7813ea-f75b-4f30-b753-8eb03749f66d)
Authenticated to Subscription: MCAPS-Hybrid-REQ-116308-2025-nitishchawla (aaf69eac-3c5e-4df0-824a-8daed137f8a2)


In [15]:
project_client = AIProjectClient(endpoint=project_endpoint, credential=credential)

list(project_client.agents.list_agents())

[{'id': 'asst_h7fE6wi7pIlSiF5rl3XAon7x', 'object': 'assistant', 'created_at': 1754184264, 'name': 'test-agent', 'description': None, 'model': 'gpt-4.1-mini', 'instructions': '\nA Q&A agent that can answer questions about the Earth at night.\nSources have a JSON format with a ref_id that must be cited in the answer using the format [ref_id].\nIf you do not have the answer, respond with "I don\'t know".\n', 'tools': [], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {}, 'metadata': {}, 'response_format': 'auto'},
 {'id': 'asst_Jj13d9kq8gmoGiYUMJCxhgVX', 'object': 'assistant', 'created_at': 1754183699, 'name': 'test-agent', 'description': None, 'model': 'gpt-4o', 'instructions': '\nA Q&A agent that can answer questions about the Earth at night.\nSources have a JSON format with a ref_id that must be cited in the answer using the format [ref_id].\nIf you do not have the answer, respond with "I don\'t know".\n', 'tools': [], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {}, 'metad

In [16]:
instructions = """
A Q&A agent that can answer questions about the Earth at night.
Sources have a JSON format with a ref_id that must be cited in the answer using the format [ref_id].
If you do not have the answer, respond with "I don't know".
"""
agent = project_client.agents.create_agent(
    model=agent_model,
    name=agent_name,
    instructions=instructions
)

print(f"AI agent '{agent_name}' created or updated successfully")

AI agent 'test-agent' created or updated successfully


In [17]:
# print("Connected to endpoint:", project_endpoint)
# print("Available agents:", len(list(project_client.agents.list_agents())))
# thread = project_client.agents.threads.create()
# retrieval_results = {}

# def agentic_retrieval() -> str:
#     """
#         Searches a NASA e-book about images of Earth at night and other science related facts.
#         The returned string is in a JSON format that contains the reference id.
#         Be sure to use the same format in your agent's response
#         You must refer to references by id number
#     """
#     # Take the last 5 messages in the conversation
#     messages = project_client.agents.messages.list(thread.id, limit=5, order=ListSortOrder.DESCENDING)
#     # Reverse the order so the most recent message is last
#     messages = list(messages)
#     messages.reverse()
#     retrieval_result = agent_client.retrieve(
#         retrieval_request=KnowledgeAgentRetrievalRequest(
#             messages=[KnowledgeAgentMessage(role=msg["role"], content=[KnowledgeAgentMessageTextContent(text=msg.content[0].text)]) for msg in messages if msg["role"] != "system"],
#             target_index_params=[KnowledgeAgentIndexParams(index_name=index_name, reranker_threshold=2.5)]
#         )
#     )

#     # Associate the retrieval results with the last message in the conversation
#     last_message = messages[-1]
#     retrieval_results[last_message.id] = retrieval_result

#     # Return the grounding response to the agent
#     return retrieval_result.response[0].content[0].text


# functions = FunctionTool({ agentic_retrieval })
# toolset = ToolSet()
# toolset.add(functions)
# project_client.agents.enable_auto_function_calls(toolset)

In [18]:
agent_client = KnowledgeAgentRetrievalClient(endpoint=endpoint, agent_name=agent_name, credential=new_credential)

thread = project_client.agents.threads.create()
retrieval_results = {}

def agentic_retrieval() -> str:
    """
        Searches a NASA e-book about images of Earth at night and other science related facts.
        The returned string is in a JSON format that contains the reference id.
        Be sure to use the same format in your agent's response
        You must refer to references by id number
    """
    # Take the last 5 messages in the conversation
    messages = project_client.agents.messages.list(thread.id, limit=5, order=ListSortOrder.DESCENDING)
    # Reverse the order so the most recent message is last
    messages = list(messages)
    messages.reverse()
    retrieval_result = agent_client.retrieve(
        retrieval_request=KnowledgeAgentRetrievalRequest(
            messages=[KnowledgeAgentMessage(role=msg["role"], content=[KnowledgeAgentMessageTextContent(text=msg.content[0].text)]) for msg in messages if msg["role"] != "system"],
            target_index_params=[KnowledgeAgentIndexParams(index_name=index_name, reranker_threshold=2.5)]
        )
    )

    # Associate the retrieval results with the last message in the conversation
    last_message = messages[-1]
    retrieval_results[last_message.id] = retrieval_result

    # Return the grounding response to the agent
    return retrieval_result.response[0].content[0].text


functions = FunctionTool({ agentic_retrieval })
toolset = ToolSet()
toolset.add(functions)
project_client.agents.enable_auto_function_calls(toolset)

In [19]:
message = project_client.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content="""
        Why do suburban belts display larger December brightening than urban cores even though absolute light levels are higher downtown?
        Why is the Phoenix nighttime street grid is so sharply visible from space, whereas large stretches of the interstate between midwestern cities remain comparatively dim?
    """
)

run = project_client.agents.runs.create_and_process(
    thread_id=thread.id,
    agent_id=agent.id,
    tool_choice=AgentsNamedToolChoice(type=AgentsNamedToolChoiceType.FUNCTION, function=FunctionName(name="agentic_retrieval")),
    toolset=toolset)
if run.status == "failed":
    raise RuntimeError(f"Run failed: {run.last_error}")
output = project_client.agents.messages.get_last_message_text_by_role(thread_id=thread.id, role="assistant").text.value

print("Agent response:", output.replace(".", "\n"))

Agent response: Suburban belts display larger December brightening than urban cores because of increased holiday light displays and activities during Christmas and New Year's
 This results in a 20 to 50 percent increase in nighttime brightness in suburban areas, which can be more noticeable despite the absolute light levels being higher downtown
 The holiday lights in suburban areas often stand out due to residential decorations, while urban cores already have consistently high light levels year-round, so the relative increase is smaller there [ref_id=0]


Regarding Phoenix, the nighttime street grid is sharply visible from space because the city is laid out along a regular grid of city blocks and streets with extensive street lighting illuminating that grid pattern
 The bright nodes at major intersections, combined with well-lit transportation corridors like Grand Avenue and many suburban municipalities linked by a network of streets and freeways, make the grid distinct and prominent 

In [20]:
import json

retrieval_result = retrieval_results.get(message.id)
if retrieval_result is None:
    raise RuntimeError(f"No retrieval results found for message {message.id}")

print("Retrieval activity")
print(json.dumps([activity.as_dict() for activity in retrieval_result.activity], indent=2))
print("Retrieval results")
print(json.dumps([reference.as_dict() for reference in retrieval_result.references], indent=2))

Retrieval activity
[
  {
    "id": 0,
    "type": "ModelQueryPlanning",
    "input_tokens": 1383,
    "output_tokens": 555
  },
  {
    "id": 1,
    "type": "AzureSearchQuery",
    "target_index": "earth_at_night",
    "query": {
      "search": "Why do suburban areas show greater December brightening in nighttime lights compared to urban cores despite higher absolute light levels downtown?"
    },
    "query_time": "2025-08-03T01:29:35.112Z",
    "count": 1,
    "elapsed_ms": 612
  },
  {
    "id": 2,
    "type": "AzureSearchQuery",
    "target_index": "earth_at_night",
    "query": {
      "search": "Why is the Phoenix nighttime street grid so sharply visible from space, while large stretches of interstate highways between Midwestern cities appear comparatively dim?"
    },
    "query_time": "2025-08-03T01:29:35.343Z",
    "count": 2,
    "elapsed_ms": 222
  },
  {
    "id": 3,
    "type": "AzureSearchSemanticRanker",
    "input_tokens": 48617
  }
]
Retrieval results
[
  {
    "type"

In [21]:
message = project_client.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content="How do I find lava at night? Use the retrieval tool to answer this question."
)

run = project_client.agents.runs.create_and_process(
    thread_id=thread.id,
    agent_id=agent.id,
    tool_choice=AgentsNamedToolChoice(type=AgentsNamedToolChoiceType.FUNCTION, function=FunctionName(name="agentic_retrieval")),
    toolset=toolset)
if run.status == "failed":
    raise RuntimeError(f"Run failed: {run.last_error}")
output = project_client.agents.messages.get_last_message_text_by_role(thread_id=thread.id, role="assistant").text.value


print("Agent response:", output.replace(".", "\n"))

Agent response: To find lava at night, you can look for the glowing light emitted by active lava flows on volcanoes, which is visible from space
 This natural light shows up as bright red or orange spots different from surrounding city lights or other features
 For example, nighttime satellite imagery of Mount Etna in Italy and Kilauea in Hawaii shows glowing lava flows clearly visible against darker surroundings
 With enough moonlight, these lava flows can be tracked and monitored for scientific research and public safety
 Specialized satellite sensors like VIIRS capture thermal and infrared data that can highlight very hot lava and cooling flows even at night [ref_id=0,1,2,3,4]

